In [1]:
import time
import sys
sys.path.append('../')
import librairies.dagfeaturingfx 
from  librairies.dagfeaturingfx import *
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import pyttsx3
from sklearn.preprocessing import MinMaxScaler
from librairies.bt import *
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax
import colorama as col
from collections import Counter

scaler = MinMaxScaler(feature_range=(0, 1))
engine = pyttsx3.init()
engine.say("Librairies loaded")
engine.runAndWait()

Importing Librairies...


In [2]:
###### RELOAD LIBRAIRIE #####
from importlib import reload
sys.path.append('../')
librairies.dagfeaturingfx= reload(librairies.dagfeaturingfx)
from  librairies.dagfeaturingfx import *

In [3]:
####### CHOIX DE LA TIME FRAME
_period = 'm15'

In [4]:
%%time
##### Récupération de la liste des tickers à partir du répertoire où sont rangées les bases
TICKER_LIST = get_ticker_list()



Récupération des tickers
CPU times: user 713 µs, sys: 914 µs, total: 1.63 ms
Wall time: 821 µs


In [5]:
%%time
engine.say("Processing featuring of dataframes, weekly, daily and intra-day")
engine.runAndWait()
##### Récupération des data pour tous les tickers sur la période demandée en intraday => df_all
df_all = get_all_data(TICKER_LIST,_period=_period)
engine.say("The job is done")
engine.runAndWait()

100%|██████████| 42/42 [00:00<00:00, 116972.62it/s]
Récupération des data intraday

CPU times: user 2.64 s, sys: 2.67 s, total: 5.31 s
Wall time: 2min 4s


In [6]:
%%time
##### Si la période demandée est déjà H1, on peut construire directement la base daily pour tous les tickers => daily_all
if _period == 'H1':
    df_all = timerange1D(df_all)
    daily_all = get_daily(df_all,TICKER_LIST) 

##### Si la période n'est pas H1, on récupère d'abord les data en 1H pour tous les tickers, et on construit la base daily à partir du 1H => daily_all
else:
    df_all = timerange1D(df_all)
    hourly_all = get_all_data(TICKER_LIST,_period='H1')
    hourly_all = timerange1D(hourly_all)
    daily_all = get_daily(hourly_all,TICKER_LIST)
    del hourly_all
engine.say("The job is done")
engine.runAndWait()


Ajout Date
100%|██████████| 42/42 [00:00<00:00, 141494.59it/s]
Récupération des data intraday


Ajout Date
100%|██████████| 42/42 [00:00<00:00, 147909.96it/s]
Récupération des data daily

CPU times: user 1min 36s, sys: 3.76 s, total: 1min 40s
Wall time: 2min 18s


In [7]:
%%time
##### A partir de la base daily_all on construit la base weekly pour tous les tickers => weekly_all
daily_all = timerange1W(daily_all)
weekly_all = get_weekly(daily_all,TICKER_LIST)
engine.say("The job is done")
engine.runAndWait()


Ajout colonne 'Date dans le weekly
100%|██████████| 42/42 [00:00<00:00, 227011.30it/s]
Récupération des tickers weekly

CPU times: user 5.27 s, sys: 488 ms, total: 5.76 s
Wall time: 10.6 s


In [10]:
%%time
##### On calcule l'ADR sur le daily
daily_all = adr(daily_all,_window=14)
engine.say("The job is done")
engine.runAndWait()

 12%|█▏        | 5/42 [00:00<00:00, 47.85it/s]
Calcul du ADR
100%|██████████| 42/42 [00:00<00:00, 42.61it/s]
CPU times: user 930 ms, sys: 79.3 ms, total: 1.01 s
Wall time: 4.06 s


In [11]:
%%time
##### On récupère l'ADR qui a été calculé en daily (daily_all) pour le mettre dans la base intraday df_all
df_all = getadr(daily_all,df_all,TICKER_LIST)

100%|██████████| 42/42 [00:00<00:00, 135508.28it/s]
Récupération de l'ADR en cours...

CPU times: user 2min 27s, sys: 14.1 s, total: 2min 42s
Wall time: 4min 30s


In [12]:
%%time
##### On calcule l'ADR High & Low dans notre base intraday df_all
df_all = adrhnl(daily_all,df_all,TICKER_LIST)


Calcul du ADR High & Low
En cours...
100%|██████████| 42/42 [00:00<00:00, 123882.40it/s]
CPU times: user 31.5 s, sys: 15.8 s, total: 47.3 s
Wall time: 2min 51s


In [13]:
%%time
##### Calcul d'une SMA 200 sur df_all
df_all = sma(df_all=df_all,_window=200)


Calcul SMA
CPU times: user 3.93 s, sys: 824 ms, total: 4.75 s
Wall time: 4.93 s


In [14]:
%%time
##### Calcul des bollinger sur df_all
df_all = bollinger(df_all,_slow=20)


Calcul BOLLINGER
CPU times: user 10.3 s, sys: 5.35 s, total: 15.7 s
Wall time: 24 s


In [15]:
%%time
##### Calcul du stochastic slow. Par défaut les paramètres sont 5 et 3.
df_all = slowstochastic(df_all,TICKER_LIST)

100%|██████████| 42/42 [00:00<00:00, 122845.72it/s]
Calcul du Slow STOCHASTIC
Fenêtre de Glissement : 5 periodes.
Période de lissage : 3 periodes.

CPU times: user 23 s, sys: 15 s, total: 38 s
Wall time: 38.3 s


In [16]:
%%time
df_all = ema(df_all,21,TICKER_LIST)

100%|██████████| 42/42 [00:00<00:00, 63344.40it/s]
Calcul de l'EMA_21en cours


Processus de calcul de l'EMA_21terminé
CPU times: user 24 s, sys: 19.2 s, total: 43.2 s
Wall time: 46.5 s


In [17]:
%%time
df_all = ema(df_all,8,TICKER_LIST)

100%|██████████| 42/42 [00:00<00:00, 128961.03it/s]
Calcul de l'EMA_8en cours


Processus de calcul de l'EMA_8terminé
CPU times: user 24.5 s, sys: 21.7 s, total: 46.2 s
Wall time: 49.6 s


In [18]:
%%time
weekly_all = pivot(weekly_all,TICKER_LIST)

100%|██████████| 42/42 [00:00<00:00, 205795.29it/s]
Calcul des PIVOT, RESISTANCE ET SUPPORT

CPU times: user 239 ms, sys: 37.5 ms, total: 277 ms
Wall time: 319 ms


In [19]:
%%time
df_all = pivotimportdf(df_all,weekly_all,TICKER_LIST)

100%|██████████| 42/42 [00:00<00:00, 256047.63it/s]
CPU times: user 26.3 s, sys: 55 s, total: 1min 21s
Wall time: 1min 21s


In [20]:
%%time
df_all = atr(df_all,TICKER_LIST,14)


Calcul ATR
100%|██████████| 42/42 [00:00<00:00, 228483.49it/s]
CPU times: user 38.1 s, sys: 3min 2s, total: 3min 40s
Wall time: 8min 37s


In [21]:
%%time
df_all = rvi(df_all,TICKER_LIST,_window=14)

100%|██████████| 42/42 [00:00<00:00, 111423.64it/s]
Calcul du RVI

CPU times: user 26.5 s, sys: 58.6 s, total: 1min 25s
Wall time: 1min 26s


In [22]:
%%time
df_all = sbgamma(df_all,TICKER_LIST)

100%|██████████| 42/42 [00:00<00:00, 216413.72it/s]
CPU times: user 28.3 s, sys: 1min 2s, total: 1min 31s
Wall time: 1min 31s


In [23]:
%%time
df_all = onhisma(df_all,TICKER_LIST,_window=5)
df_all = onlosma(df_all,TICKER_LIST,_window=5)

100%|██████████| 42/42 [00:00<00:00, 156587.35it/s]
Calcul ONHISMA
On High Simple Moving Average Calculation

100%|██████████| 42/42 [00:00<00:00, 161467.25it/s]
Calcul ONLOSMA
On High Simple Moving Average Calculation

CPU times: user 57.4 s, sys: 1min 54s, total: 2min 51s
Wall time: 2min 52s


In [24]:
%%time
df_all = onhisma(df_all,TICKER_LIST,_window=21)
df_all = onlosma(df_all,TICKER_LIST,_window=21)

100%|██████████| 42/42 [00:00<00:00, 201096.77it/s]
Calcul ONHISMA
On High Simple Moving Average Calculation

100%|██████████| 42/42 [00:00<00:00, 162659.99it/s]
Calcul ONLOSMA
On High Simple Moving Average Calculation

CPU times: user 59.9 s, sys: 2min 15s, total: 3min 15s
Wall time: 3min 16s


In [25]:
%%time
df_all = onhisma(df_all,TICKER_LIST,_window=34)
df_all = onlosma(df_all,TICKER_LIST,_window=34)

100%|██████████| 42/42 [00:00<00:00, 161024.47it/s]
Calcul ONHISMA
On High Simple Moving Average Calculation

100%|██████████| 42/42 [00:00<00:00, 119350.11it/s]
Calcul ONLOSMA
On High Simple Moving Average Calculation

CPU times: user 1min, sys: 2min 30s, total: 3min 30s
Wall time: 3min 31s


In [26]:
%%time
df_all = importohlc(df_all,weekly_all,TICKER_LIST,_suffix='_weekly')

100%|██████████| 42/42 [00:00<00:00, 129529.98it/s]Récupération des OHLC en cours...

CPU times: user 32.9 s, sys: 1min 8s, total: 1min 41s
Wall time: 1min 42s


In [27]:
%%time
df_all = importohlc(df_all=df_all,other_all=daily_all,TICKER_LIST=TICKER_LIST,_suffix='_daily')

100%|██████████| 42/42 [00:00<00:00, 139699.26it/s]Récupération des OHLC en cours...

CPU times: user 32.9 s, sys: 1min 24s, total: 1min 56s
Wall time: 1min 57s


In [30]:
%%time
joblib.dump(df_all,'JOBLIB/Built_bases/df_all')
joblib.dump(daily_all,'JOBLIB/Built_bases/daily_all')
joblib.dump(weekly_all,'JOBLIB/Built_bases/weekly_all')

CPU times: user 1.78 s, sys: 2.32 s, total: 4.09 s
Wall time: 4.56 s


['JOBLIB/Built_bases/weekly_all']